In [1]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
# import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
# import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
dummy = pd.read_csv('data/book_review_labelled_data.csv')[0:3]

In [3]:
text_docs = dummy.reviewText
text_docs[0]
tk = WordPunctTokenizer() 
def tkze(tk,text_docs): 
    return [" ".join(tk.tokenize(doc)) for doc in text_docs]

docs = tkze(tk,text_docs)

In [4]:
# number of words 
def get_nb_words(docs): 
    return ([len(doc.split()) for doc in docs])

get_nb_words(docs)

[363, 288, 28]

In [30]:
# average word size 
def get_av_word_size(docs):
    av_words_size_per_doc = [] 
    for doc in docs: 
        words_size = [len(word) for word in doc.split()]
        av_size = np.mean(np.array(words_size))
        av_words_size_per_doc.append(av_size)
    return av_words_size_per_doc
get_av_word_size(docs)  

[4.223140495867769, 4.399305555555555, 3.2857142857142856]

In [39]:
# number of unique words (proxy for how rich is the reviewer vocabulary)
# not really sure if this feature will be useful (maybe couple with PoS) 
def get_nb_unique_words(docs):
    unique_words = [len(set(word for word in doc)) for doc in docs] 
    normalized = (np.array(unique_words)/np.array(get_nb_words(docs))).tolist()
    return unique_words, normalized
get_nb_unique_words(docs)


([60, 45, 24], [0.1652892561983471, 0.15625, 0.8571428571428571])

In [31]:
# important for spacy related functions
nlp = spacy.load('en_core_web_sm')
nlp_docs = list(tqdm(nlp.pipe(docs), total=len(docs)))

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 27.25it/s]


In [35]:
# getting NERs (proxy for attention to detail by reviewers that describe book parts, e.g. naming book charters)
def get_most_common_persons(nlp_docs, n_most_common=10): 
    matcher = Matcher(nlp.vocab)
    pattern = [[{'ENT_TYPE':'PERSON'}]]
    matcher.add('ppl', pattern)

    persons = []
    for doc in nlp_docs: 
        matches = matcher(doc)
        for match_id, start, end in matches: 
            persons.append(str(doc[start:end]))

    most_common_ents = [person for person in Counter(persons).most_common()[:n_most_common]]
    return most_common_ents

get_most_common_persons(nlp_docs)    

[('Churchill', 4),
 ('Gladstone', 1),
 ('Jenkins', 1),
 ('Erlendur', 1),
 ('Sveinsson', 1),
 ('Skarphedinn', 1),
 ('Sigurdur', 1),
 ('Oli', 1),
 ('Eva', 1),
 ('Lind', 1)]

In [37]:
# number of predifined parts of speech 
def get_nb_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN','VERB'] 
    nb_PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc if token.pos_ in PoS_list]  
        nb_in_doc = sum([Counter(total_PoS)[x] for x in PoS_list])
        nb_PoS.append(nb_in_doc)
    return nb_PoS

get_nb_PoS(nlp_docs)

[138, 109, 11]

In [85]:
def get_nb_unique_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN'] 
    nb_unique_PoS = [] 
    nb_PoS = []
    word4PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc]
        mask = np.array([True if PoS in PoS_list else False for PoS in total_PoS])
        # print(mask[:10])
        words = np.array([token.text for token in doc])
        words_under_selected_PoS = words[mask].tolist()
        word4PoS.append(words_under_selected_PoS)
        # print(words_under_selected_PoS)
        unique_words = get_nb_unique_words(words_under_selected_PoS)[0]
        nb_unique_PoS.append(len(unique_words))
        nb_PoS.append(len(words_under_selected_PoS))
        
    ratio = np.array(nb_unique_PoS)/ np.array(nb_PoS)
    return ratio.tolist(), word4PoS
    
get_nb_unique_PoS(nlp_docs)

([1.0, 1.0, 1.0],
 [['history',
   'professor',
   'author',
   'acclaimed',
   'bio',
   'fine',
   'biography',
   'greatest',
   '20th',
   'century',
   'figure',
   'own',
   'experiences',
   'political',
   'fortunes',
   'maneuverings',
   'American',
   'reader',
   'slow',
   'odd',
   'political',
   'system',
   'politicians',
   'district',
   'traditional',
   'public',
   'political',
   'bio',
   'psychoanalysis',
   'much',
   'personal',
   'life',
   'moves',
   'good',
   'clip',
   'pages',
   'Jenkins',
   'living',
   'writer',
   'contracts',
   'schedules',
   'royalties',
   'politics',
   'avocation',
   'author',
   'fond',
   'unusual',
   'words',
   'quot',
   'rumbustious',
   'other',
   'hand',
   'wit',
   'dry',
   'evidence',
   'hardcover',
   'edition',
   'clean',
   'halfway',
   'point',
   'quot',
   'quot',
   'quot',
   'quot',
   'quot',
   'opposition',
   'similar',
   'infelicities',
   'nice',
   'balance',
   'healthy',
   'respect',
 

In [ ]:
def get_nb_unique_PoS(docs):
    unique_words = [len(set(word for word in doc)) for doc in docs] 
    normalized = (np.array(unique_words)/np.array(get_nb_words(docs))).tolist()
    return unique_words, normalized